In [4]:
# Importaciones necesarias
import pandas as pd 
import numpy as np
from word2number import w2n
pd.set_option('display.max_columns', None)


# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
#guardamos despues de arreglar las columnas
df_data2 = pd.read_csv("../DATA/data_fase_2.csv", index_col= 0)

# -----------------------------------------------------------------------------------------

# Gestion de Nulos

In [6]:
# calculamos el % de nulos por columna
df_nulos = pd.DataFrame((df_data2.isnull().sum() / df_data2.shape[0]) * 100, columns = ["%_nulos"])
# filtramos el DataFrame para quedarnos solo con aquellas columnas que tengan nulos
df_nulos[df_nulos["%_nulos"] > 0]

,%_nulos
BUSINESSTRAVEL,47.831475
DAILYRATE,7.682776
DEPARTMENT,81.288724
EDUCATIONFIELD,46.158612
EMPLOYEENUMBER,26.703841
HOURLYRATE,5.204461
MARITALSTATUS,40.334572
MONTHLYINCOME,52.230483
OVERTIME,41.883519
PERFORMANCERATING,12.081784


## Nulos categoricas

In [7]:
# sacamos una lista de las variables categoricas que tienen nulos
nulos_esta_cat = df_data2[df_data2.columns[df_data2.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['BUSINESSTRAVEL', 'DEPARTMENT', 'EDUCATIONFIELD', 'MARITALSTATUS'], dtype='object')


In [9]:
# % de nulos que tenemos en cada una de las columnas anteriores
display(df_data2[nulos_esta_cat].isnull().sum() / df_data2.shape[0])

BUSINESSTRAVEL    0.478315
DEPARTMENT        0.812887
EDUCATIONFIELD    0.461586
MARITALSTATUS     0.403346
dtype: float64

In [ ]:
# % de nulos por categoria de cada columna
for col in nulos_esta_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df_data2[col].value_counts() / df_data2.shape[0])
    print("........................")

La distribución de las categorías para la columna BUSINESSTRAVEL


BUSINESSTRAVEL
travel rarely        0.363073
travel frequently    0.102230
non-travel           0.056382
Name: count, dtype: float64

........................
La distribución de las categorías para la columna DEPARTMENT


DEPARTMENT
research & development    0.121437
sales                     0.056382
human resources           0.009294
Name: count, dtype: float64

........................
La distribución de las categorías para la columna EDUCATIONFIELD


EDUCATIONFIELD
life sciences       0.216233
medical             0.171004
marketing           0.064436
technical degree    0.042751
other               0.036555
human resources     0.007435
Name: count, dtype: float64

........................
La distribución de las categorías para la columna MARITALSTATUS


MARITALSTATUS
married     0.271995
single      0.201363
divorced    0.123296
Name: count, dtype: float64

........................


In [ ]:
# Creamos listas en funcion del metodo que vayamos a aplicar
# decidimos sustituir los nulos con una nueva categoria "Unknown" porque en inguna la moda 
# esta muy por encima del resto

lista_unknown = ["BUSINESSTRAVEL", "DEPARTMENT", "EDUCATIONFIELD", "MARITALSTATUS"]

In [ ]:
## Comprobamos que ya no haya nulos
for columna in lista_unknown:
    df_data2[columna] = df_data2[columna].fillna("Unknown")
    
df_data2[lista_unknown].isnull().sum()

BUSINESSTRAVEL    0
DEPARTMENT        0
EDUCATIONFIELD    0
MARITALSTATUS     0
dtype: int64

# Nulos numericas

In [ ]:
# sacamos una lista de las variables numericas que tienen nulos
nulos_esta_num = df_data2[df_data2.columns[df_data2.isnull().any()]].select_dtypes(include = np.number).columns
print("Las columnas numéricas que tienen nulos son : \n ")
print(nulos_esta_num)

Las columnas numéricas que tienen nulos son : 
 
Index(['DAILYRATE', 'EMPLOYEENUMBER', 'HOURLYRATE', 'MONTHLYINCOME',
       'OVERTIME', 'PERFORMANCERATING', 'TOTALWORKINGYEARS',
       'WORKLIFEBALANCE'],
      dtype='object')


In [ ]:
# nulos que tenemos en cada una de las columnas numericas
df_data2[nulos_esta_num].isnull().sum() / df_data2.shape[0]

DAILYRATE            0.076828
EMPLOYEENUMBER       0.267038
HOURLYRATE           0.052045
MONTHLYINCOME        0.522305
OVERTIME             0.418835
PERFORMANCERATING    0.120818
TOTALWORKINGYEARS    0.325898
WORKLIFEBALANCE      0.066914
dtype: float64

In [ ]:
df_data2["WORKLIFEBALANCE"].value_counts()

WORKLIFEBALANCE
3.0    913
2.0    359
4.0    155
1.0     79
Name: count, dtype: int64

In [ ]:
df_data2.head()

,AGE,ATTRITION,BUSINESSTRAVEL,DAILYRATE,DEPARTMENT,DISTANCEFROMHOME,EDUCATION,EDUCATIONFIELD,EMPLOYEENUMBER,ENVIRONMENTSATISFACTION,GENDER,HOURLYRATE,JOBINVOLVEMENT,JOBLEVEL,JOBROLE,JOBSATISFACTION,MARITALSTATUS,MONTHLYINCOME,MONTHLYRATE,NUMCOMPANIESWORKED,OVERTIME,PERCENTSALARYHIKE,PERFORMANCERATING,RELATIONSHIPSATISFACTION,STOCKOPTIONLEVEL,TOTALWORKINGYEARS,TRAININGTIMESLASTYEAR,WORKLIFEBALANCE,YEARSATCOMPANY,YEARSSINCELASTPROMOTION,YEARSWITHCURRMANAGER,REMOTEWORK
0,51.0,0,Unknown,684.0,Unknown,6,3,Unknown,162.0,1,male,51.0,3,5,research director,3,Unknown,19537.0,6462,7,0.0,13,3.0,3,0,NaN,5,3.0,20,15,15,1
1,52.0,0,Unknown,699.0,Unknown,1,4,life sciences,259.0,3,male,65.0,2,5,manager,3,Unknown,19999.0,5678,0,NaN,14,3.0,1,1,34.0,5,3.0,33,11,9,1
2,42.0,0,travel rarely,532.0,research & development,4,2,technical degree,319.0,3,male,58.0,3,5,manager,4,married,19232.0,4933,1,0.0,11,3.0,4,0,22.0,3,NaN,22,11,15,1
3,47.0,0,travel rarely,359.0,Unknown,2,4,medical,NaN,1,female,82.0,3,4,research director,3,married,17169.0,26703,3,NaN,19,3.0,2,2,NaN,2,NaN,20,5,6,0
4,46.0,0,Unknown,1319.0,Unknown,3,3,technical degree,NaN,1,female,45.0,4,4,sales executive,1,divorced,NaN,7739,2,0.0,12,3.0,4,1,NaN,5,3.0,19,2,8,0


In [ ]:
#EMPLOYEE NUMBER ES UN ID DE EMPLEADO, HACEMOS NUEVA COL CON RESET INDEX
#HOURLY RATE PREGUNTAR POR QUE SALARIO X HORA X 8 NO CUADRA CON SALARIO MENSUAL
#PREGUNTAR INCOME Y RATE ES LO MISMO??

In [ ]:
# Realizar copia DF
df_copia_ejercicios = df_data2.copy()

In [ ]:
# Imputaremos los nulos de estas columnas
lista_num = ["DAILYRATE", "PERFORMANCERATING", "TOTALWORKINGYEARS"]  

In [ ]:
# Reemplazaremos por "Unknown" los nulos de estas columnas
lista_unknown = ["EMPLOYEENUMBER"]

In [ ]:
# Reemplazaremos por la moda los nulos de estas columnas (porque la moda es muy superior al resto de categorias)
lista_moda = ["WORKLIFEBALANCE"]

### Metodo IterativeImputer

In [ ]:
clase_imputer = IterativeImputer(max_iter = 20, random_state = 42)

transformamos= clase_imputer.fit_transform(df_copia_ejercicios[lista_num])

transformamos

array([[ 684.        ,    3.        ,   11.22316701],
       [ 699.        ,    3.        ,   34.        ],
       [ 532.        ,    3.        ,   22.        ],
       ...,
       [ 903.        ,    3.        ,    9.        ],
       [1229.        ,    3.14616004,   12.        ],
       [ 566.        ,    3.        ,   11.1319115 ]])

In [ ]:
# creamos una nueva lista con el nombre que pondremos a las columnas que salen de "IterativeImputer"
lista_iterative = []
for col in lista_num:
    col = col + "_iterative" # añade "_iterative" al nombre original
    lista_iterative.append(col)

lista_iterative

['DAILYRATE_iterative',
 'PERFORMANCERATING_iterative',
 'TOTALWORKINGYEARS_iterative']

In [ ]:
# Añadimos al df copia las columnas nuevas de IterativeImputer
df_copia_ejercicios[lista_iterative] = transformamos
print(df_copia_ejercicios[lista_iterative].isnull().sum()) #comprobamos que no tienen nulos despues de la imputacion

DAILYRATE_iterative            0
PERFORMANCERATING_iterative    0
TOTALWORKINGYEARS_iterative    0
dtype: int64


### Metodo KNNimputer

In [ ]:
imputer_knn_ejercicios = KNNImputer(n_neighbors = 5)

imputer_knn_imputado_ejer = imputer_knn_ejercicios.fit_transform(df_copia_ejercicios[lista_num])

imputer_knn_imputado_ejer

array([[ 684. ,    3. ,   11.4],
       [ 699. ,    3. ,   34. ],
       [ 532. ,    3. ,   22. ],
       ...,
       [ 903. ,    3. ,    9. ],
       [1229. ,    3.2,   12. ],
       [ 566. ,    3. ,   12.2]])

In [ ]:
# creamos una nueva lista con el nombre que pondremos a las columnas que salen de "KNNImputer"
lista_knn = []
for col in lista_num:
    col = col + "_knn" # añade "_knn" al nombre original
    lista_knn.append(col)

lista_knn    

['DAILYRATE_knn', 'PERFORMANCERATING_knn', 'TOTALWORKINGYEARS_knn']

In [ ]:
# Añadimos al df copia las columnas nuevas de KNNImputer
df_copia_ejercicios[lista_knn] = imputer_knn_imputado_ejer
# comprobamos los nulos
print(df_copia_ejercicios[lista_knn].isnull().sum())

DAILYRATE_knn            0
PERFORMANCERATING_knn    0
TOTALWORKINGYEARS_knn    0
dtype: int64


# Comprobamos ambos metodos VS el original

In [ ]:
# creamos una nueva lista con las columnas originales que hemos imputado y las columnas generadas por IterativeImputer y KNNImputer
lista_describe = []
for i in range(len(lista_num)):
    lista_describe.append(lista_num[i])
    lista_describe.append(lista_iterative[i])
    lista_describe.append(lista_knn[i])

In [ ]:
#para comparar y decidir con cual nos quedamos mostramos un describe 
df_copia_ejercicios.describe()[lista_describe]

,DAILYRATE,DAILYRATE_iterative,DAILYRATE_knn,PERFORMANCERATING,PERFORMANCERATING_iterative,PERFORMANCERATING_knn,TOTALWORKINGYEARS,TOTALWORKINGYEARS_iterative,TOTALWORKINGYEARS_knn
count,1490.000000,1614.000000,1614.000000,1419.00000,1614.000000,1614.000000,1088.000000,1614.000000,1614.000000
mean,802.085235,802.074399,807.967993,3.15081,3.150809,3.158986,11.318934,11.314480,11.204756
std,403.246954,387.460081,392.502027,0.35799,0.335658,0.341630,7.727675,6.346017,6.643326
min,103.000000,103.000000,103.000000,3.00000,3.000000,3.000000,0.000000,0.000000,0.000000
25%,468.250000,498.000000,495.250000,3.00000,3.000000,3.000000,6.000000,8.000000,7.000000
50%,798.000000,801.987915,808.000000,3.00000,3.000000,3.000000,10.000000,11.004695,10.000000
75%,1157.000000,1130.750000,1136.750000,3.00000,3.143232,3.000000,15.000000,11.798927,14.000000
max,1499.000000,1499.000000,1499.000000,4.00000,4.000000,4.000000,40.000000,40.000000,40.000000


In [ ]:
print(lista_describe)

['DAILYRATE', 'DAILYRATE_iterative', 'DAILYRATE_knn', 'PERFORMANCERATING', 'PERFORMANCERATING_iterative', 'PERFORMANCERATING_knn', 'TOTALWORKINGYEARS', 'TOTALWORKINGYEARS_iterative', 'TOTALWORKINGYEARS_knn']


In [ ]:
## Seleccionamos con que metodo nos quedamos y eliminamos columnas

df_copia_ejercicios.drop(['DAILYRATE', 'DAILYRATE_knn', 'PERFORMANCERATING', 'PERFORMANCERATING_knn', 'TOTALWORKINGYEARS', 'TOTALWORKINGYEARS_knn'], axis = 1, inplace = True)

# Key valor actual, Value nuevo nombre (nombre que tenian antes para que cuadre con la documentacion)
nuevo_nombre = {'DAILYRATE_iterative' : 'DAILYRATE',
 'PERFORMANCERATING_iterative' : 'PERFORMANCERATING',
 'TOTALWORKINGYEARS_iterative' : 'TOTALWORKINGYEARS'}
df_copia_ejercicios.rename(columns = nuevo_nombre, inplace = True)



In [ ]:
## Para los que creemos una categoria nueva "Desconocido"

df_copia_ejercicios["EMPLOYEENUMBER"] = df_copia_ejercicios["EMPLOYEENUMBER"].fillna("Unknown")
    
df_copia_ejercicios["EMPLOYEENUMBER"].isnull().sum()

0

In [ ]:
# para los que usamos la moda
df_copia_ejercicios["WORKLIFEBALANCE"] = df_copia_ejercicios["WORKLIFEBALANCE"].fillna(df_copia_ejercicios["WORKLIFEBALANCE"].mode()[0])
    
df_copia_ejercicios["WORKLIFEBALANCE"].isnull().sum()

0

In [ ]:
df_copia_ejercicios

,AGE,ATTRITION,BUSINESSTRAVEL,DEPARTMENT,DISTANCEFROMHOME,EDUCATION,EDUCATIONFIELD,EMPLOYEENUMBER,ENVIRONMENTSATISFACTION,GENDER,HOURLYRATE,JOBINVOLVEMENT,JOBLEVEL,JOBROLE,JOBSATISFACTION,MARITALSTATUS,MONTHLYINCOME,MONTHLYRATE,NUMCOMPANIESWORKED,OVERTIME,PERCENTSALARYHIKE,RELATIONSHIPSATISFACTION,STOCKOPTIONLEVEL,TRAININGTIMESLASTYEAR,WORKLIFEBALANCE,YEARSATCOMPANY,YEARSSINCELASTPROMOTION,YEARSWITHCURRMANAGER,REMOTEWORK,DAILYRATE,PERFORMANCERATING,TOTALWORKINGYEARS
0,51.0,0,Unknown,Unknown,6,3,Unknown,162.0,1,male,51.0,3,5,research director,3,Unknown,19537.0,6462,7,0.0,13,3,0,5,3.0,20,15,15,1,684.0,3.00000,11.223167
1,52.0,0,Unknown,Unknown,1,4,life sciences,259.0,3,male,65.0,2,5,manager,3,Unknown,19999.0,5678,0,NaN,14,1,1,5,3.0,33,11,9,1,699.0,3.00000,34.000000
2,42.0,0,travel rarely,research & development,4,2,technical degree,319.0,3,male,58.0,3,5,manager,4,married,19232.0,4933,1,0.0,11,4,0,3,3.0,22,11,15,1,532.0,3.00000,22.000000
3,47.0,0,travel rarely,Unknown,2,4,medical,Unknown,1,female,82.0,3,4,research director,3,married,17169.0,26703,3,NaN,19,2,2,2,3.0,20,5,6,0,359.0,3.00000,10.971828
4,46.0,0,Unknown,Unknown,3,3,technical degree,Unknown,1,female,45.0,4,4,sales executive,1,divorced,NaN,7739,2,0.0,12,4,1,5,3.0,19,2,8,0,1319.0,3.00000,11.714245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1609,36.0,1,travel rarely,Unknown,3,1,life sciences,967.0,3,male,51.0,2,3,sales executive,4,married,10325.0,5518,1,NaN,11,1,1,6,3.0,16,3,7,0,530.0,3.15374,11.104071
1610,45.0,0,non-travel,Unknown,4,2,Unknown,972.0,3,male,57.0,3,2,laboratory technician,2,Unknown,4447.0,23163,1,NaN,12,2,0,5,2.0,9,0,8,1,805.0,3.00000,11.316743
1611,39.0,0,travel rarely,Unknown,13,5,Unknown,Unknown,1,male,41.0,4,3,sales executive,3,single,NaN,2560,0,0.0,18,4,0,3,3.0,8,0,7,1,903.0,3.00000,9.000000
1612,36.0,0,non-travel,Unknown,8,4,technical degree,990.0,1,male,84.0,3,2,sales executive,4,divorced,NaN,25952,4,0.0,13,4,2,3,3.0,7,0,7,0,1229.0,3.14616,12.000000


In [ ]:
df_copia_ejercicios.to_csv("../DATA/df_clean_sin_nulos.csv")